In [15]:
import os
import click
import re
import json
import tempfile
import torch
import dnnlib

from training import training_loop
from metrics import metric_main
from torch_utils import training_stats
from torch_utils import custom_ops

In [26]:
# =======================================================
# Stylegan2 configuration
# =======================================================
#    python train.py --outdir=~/training-runs \
#                    --data=~/datasets/ffhq.zip \
#                    --gpus=8 \
#                    --cfg=stylegan2 \
#                    --mirror=1 \
#                    --aug=noaug
args = dnnlib.EasyDict()
args.num_gpus = 8
args.image_snapshot_ticks = 50
args.network_snapshot_ticks = 50
args.metrics = ['fid50k_full']
args.random_seed = 0
args.training_set_kwargs = dnnlib.EasyDict(class_name='training.dataset.ImageFolderDataset', 
                                           path='~/datasets/ffhq.zip', 
                                           use_labels=True, 
                                           max_size=None, 
                                           xflip=False)
args.data_loader_kwargs = dnnlib.EasyDict(pin_memory=True, 
                                          num_workers=3, 
                                          prefetch_factor=2)
args.training_set_kwargs.resolution  = 1024


In [24]:
import pprint
pprint.pprint(args)

{'data_loader_kwargs': {'num_workers': 3,
                        'pin_memory': True,
                        'prefetch_factor': 2},
 'image_snapshot_ticks': 50,
 'metrics': ['fid50k_full'],
 'network_snapshot_ticks': 50,
 'num_gpus': 8,
 'random_seed': 0,
 'training_set_kwargs': {'class_name': 'training.dataset.ImageFolderDataset',
                         'max_size': None,
                         'path': '~/datasets/ffhq.zip',
                         'use_labels': True,
                         'xflip': False}}
